In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

**Repository Version** 
This notebook was run at commit:
``

# Input-Output Test: Positive Reflectivity Only
The previous [input-output test](./fsroot_thin_bin.ipynb) with both reflectivities found issues in describing strong waves below 1.3 GeV. 
Here we will perform a very similar study, but restrict our waveset to only the extremely dominant positive reflectivity waves, to see if our model performs better and still describes the MC data. 

As before, we integrate over the entire $-t$ range from $0.1\leq -t \leq 1.0$, with the waveset:
* $b_1(1235)$ and $\rho(1450)$ Breit-Wigners fixed to their PDG values
  * an isotropic background is included, but is so small it's negligible
  * Different from before, we fit with only positive reflectivities here
* No `OmegaDalitz` amplitudes for changing the dalitz distribution of the $\omega$ and its corresponding $\lambda$ distribution
* No $D/S$ ratio that would be typically associated with the $b_1$
* Only in the PARA 0 orientation

Fits were performed with 500 randomized fits and 100 bootstrap fits in 10 MeV mass independent bins

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager
import neutralb1.analysis.statistics as stats

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/io-tests/pos-refl/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
%%bash
# print out truth YAML file used to submit truth fits
cat $STUDY_DIR/truth_submission.YAML

In [ ]:
# load in preprocessed results
with open(f"{STUDY_DIR}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
    data = pkl.load(f)
    results = ResultManager(**data)

In [ ]:
results.summary()

## Analysis

### Standard Plots
Lets view the standard set of plots to view how our model performed overall

In [ ]:
results.plot.intensity.jp()
plt.savefig(f"{STUDY_DIR}/plots/jp.pdf")

In [ ]:
results.plot.intensity.waves()
plt.savefig(f"{STUDY_DIR}/plots/waves.pdf")

In [ ]:
results.plot.intensity.waves(fractional=True)
plt.savefig(f"{STUDY_DIR}/plots/waves_fractional.pdf")

In [ ]:
results.plot.diagnostic.matrix()
plt.savefig(f"{STUDY_DIR}/plots/matrix.pdf")

In [ ]:
results.plot.intensity.moments()
plt.savefig(f"{STUDY_DIR}/plots/moments.pdf")

In [ ]:
sig_moments = list(results.get_significant_moments(threshold=0.02))
results.plot.intensity.moments(moments=sig_moments)
plt.savefig(f"{STUDY_DIR}/plots/significant_moments.pdf")

### Additional Plots

In [ ]:
# mass-phase plot of the two strongest amplitudes that capture the b1 and rho resonances we generated
results.plot.phase.mass_phase("p1p0S", "p1mpP", {"color": "tab:blue"}, {"color": "tab:orange"})
plt.savefig(f"{STUDY_DIR}/plots/mass_phase.pdf")

A few joyplots for comparing the bootstrap distributions to the truth (bars) and nominal fit results (points)

In [ ]:
results.plot.bootstrap.joyplot(["1p", "1m"], figsize=(10, 8), overlap=1.3, truth_scaling=0.5)
plt.show()
results.plot.bootstrap.joyplot(["p1p0S", "p1mpP"], figsize=(10, 8), overlap=1.3, truth_scaling=0.5, colormap=["tab:blue", "tab:orange"])
plt.show()
phase = results.phase_difference_dict[("p1p0S", "p1mpP")]
results.plot.bootstrap.joyplot([phase], figsize=(10, 8), overlap=1.3, truth_scaling=0.5, colormap=["gray"])
plt.show()

### Statistical Tests
Lets run a couple easy statistical tests for all fit indices, to check whether our bootstrap distributions are normal and / or heavily biased

In [ ]:
sig_amplitudes = results.get_significant_amplitudes(threshold=0.05)
significant_phases = results.get_significant_phases(threshold=0.05)
columns = list(sig_amplitudes) + list(significant_phases)

if results.bootstrap_df is None: # assure type hinter that bootstrap df exists
    raise ValueError("Bootstrap dataframe is not available in results.")

fail_rates = stats.normality_test(
    results.fit_df, 
    results.bootstrap_df,
    columns,
    alpha=0.01, 
    output=f"{STUDY_DIR}/plots/normality_test.pdf",
    is_acc_corrected=results.is_acceptance_corrected
)
for col, rate in fail_rates.items():
    print(f"Normality test failure rate for {col}: {rate:.2%}")

In [ ]:
sig_amplitudes = results.get_significant_amplitudes(threshold=0.05)
significant_phases = results.get_significant_phases(threshold=0.05)
columns = list(sig_amplitudes) + list(significant_phases)

if results.bootstrap_df is None: # assure type hinter that bootstrap df exists
    raise ValueError("Bootstrap dataframe is not available in results.")
fail_rates = stats.bias_test(
    results.fit_df, 
    results.bootstrap_df, 
    columns, 
    threshold=0.1,
    output=f"{STUDY_DIR}/plots/bias_test.pdf",
    is_acc_corrected=results.is_acceptance_corrected
)
for col, rate in fail_rates.items():
    print(f"Bias test failure rate for {col}: {rate:.2%}")

In [ ]:
sig_moments = list(results.get_significant_moments(threshold=0.01))
if results.proj_moments_df is None:
    raise ValueError("Projected moments dataframe is not available in results.")
if results.bootstrap_proj_moments_df is None:
    raise ValueError("Bootstrap projected moments dataframe is not available in results.")

fail_rates = stats.normality_test(
    results.proj_moments_df,
    results.bootstrap_proj_moments_df,
    sig_moments,
    alpha=0.01,
    output=f"{STUDY_DIR}/plots/moments_normality_test.pdf",
    is_acc_corrected=results.is_acceptance_corrected
)
for col, rate in fail_rates.items():
    print(f"Normality test failure rate for {col}: {rate:.2%}")

### Comparison to Previous I/O Test

In [ ]:
# open up the old results for comparison
with open(f"{STUDY_DIR}/../thin-bins/t_0.1-1.0/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
    data_old = pkl.load(f)
    old_results = ResultManager(**data_old)